In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk


class ImageProcessorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processor")
        self.root.configure(bg="#F0F0F0")

        self.original_image = None

        self.create_widgets()

    def create_widgets(self):
        # Load Image Button
        self.load_image_button = tk.Button(
            self.root,
            text="Load Image",
            bg="#4CAF50",
            fg="white",
            font=("Arial", 12, "bold"),
            command=self.load_image,
        )
        self.load_image_button.grid(
            row=0, column=0, columnspan=5, padx=10, pady=10, sticky="ew"
        )

        # Image Display
        self.image_label = tk.Label(self.root, bg="#FFFFFF", bd=2, relief="groove")
        self.image_label.grid(
            row=1, column=0, columnspan=5, padx=10, pady=10, sticky="nsew"
        )

        # Operation Buttons
        operations = {
            "Opening": self.apply_opening,
            "Closing": self.apply_closing,
            "Hough Circle Transform": self.apply_hough_circle_transform,
            "Segmentation - Thresholding": self.apply_thresholding
        }

        row_count = 2
        column_count = 0
        for operation_name, operation_func in operations.items():
            button = tk.Button(
                self.root,
                text=operation_name,
                bg="#008CBA",
                fg="white",
                font=("Arial", 12, "bold"),
                command=operation_func,
            )
            button.grid(row=row_count, column=column_count, padx=5, pady=5, sticky="ew")
            column_count += 1
            if column_count == 5:
                column_count = 0
                row_count += 1

    def load_image(self):
        path = filedialog.askopenfilename()
        if path:
            self.original_image = cv2.imread(path)
            self.display_image(self.original_image)

    def display_image(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        self.image_label.configure(image=image)
        self.image_label.image = image

    def apply_opening(self):
        if self.original_image is not None:
            kernel_size = 5  # Kernel size for opening
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            open_image = cv2.morphologyEx(self.original_image, cv2.MORPH_OPEN, kernel)
            self.display_image(open_image)

    def apply_closing(self):
        if self.original_image is not None:
            kernel_size = 5  # Kernel size for closing
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            close_image = cv2.morphologyEx(self.original_image, cv2.MORPH_CLOSE, kernel)
            self.display_image(close_image)

    def apply_hough_circle_transform(self):
        if self.original_image is not None:
            gray_image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(
                gray_image,
                cv2.HOUGH_GRADIENT,
                dp=1,
                minDist=20,
                param1=50,
                param2=30,
                minRadius=0,
                maxRadius=0,
            )
            if circles is not None:
                circles = np.uint16(np.around(circles))
                hough_image = self.original_image.copy()
                for i in circles[0, :]:
                    cv2.circle(hough_image, (i[0], i[1]), i[2], (0, 255, 0), 2)
                    cv2.circle(hough_image, (i[0], i[1]), 2, (0, 0, 255), 3)
                self.display_image(hough_image)

    def apply_thresholding(self):
        if self.original_image is not None:
            gray_image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2GRAY)
            _, binary_image = cv2.threshold(
                gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )
            self.display_image(binary_image)

    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessorApp(root)
    app.run()